In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Dense, Activation, Dropout, Input, LSTM, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/share/software/user/open/py-scipy/1.6.3_py39/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
2025-11-30 09:39:43.316478: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-30 09:39:43.490156: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-30 09:39:48.303643: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.

## Scaling all sfe and climate data to be between 0 and 1
https://www.geeksforgeeks.org/deep-learning/long-short-term-memory-lstm-rnn-in-tensorflow/
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [9]:
# read in list of arrays for each pixel - these are fully filtered
sfe_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_filtered_3mwindow.npz", allow_pickle = True)["sfe_window"] 
climate_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_filtered_3mwindow.npz", allow_pickle = True)["climate_window"]



In [10]:
print(sfe_pre.shape)
print(climate_pre.shape)

(3836373,)
(3836373, 3, 8)


In [11]:
sfe_pre_rs = sfe_pre.reshape(-1,1) # single feature
sfe_pre_rs.shape


(3836373, 1)

In [12]:
# climate data is 3d, needs to be 2d for minmaxscaler
# so reshape and then turn back to original shape after processing
climate_pre_ogshape = climate_pre.shape
print(climate_pre_ogshape)

climate_pre_rs = climate_pre.reshape(-1, climate_pre_ogshape[2]) # (pixel-month pairs * timesteps, 8)
climate_pre_rs.shape

(3836373, 3, 8)


(11509119, 8)

In [13]:
scaler_sfe = MinMaxScaler(feature_range=(0,1))
scaler_climate = MinMaxScaler(feature_range=(0,1))

sfe_pre_scaled = scaler_sfe.fit_transform(sfe_pre_rs)
climate_pre_scaled = scaler_climate.fit_transform(climate_pre_rs)



In [14]:
print(sfe_pre_scaled.min(), sfe_pre_scaled.max())
print(climate_pre_scaled.min(), climate_pre_scaled.max())


0.0 1.0
0.0 1.0000000000000002


In [15]:
sfe_back = scaler_sfe.inverse_transform(sfe_pre_scaled)

sfe_back.shape
print(sfe_back.min(), sfe_back.max())
print(sfe_pre_rs.min(), sfe_pre_rs.max())


-0.4033722536155057 5.223653647385592
-0.40337225361550577 5.223653647385593


In [16]:
# change climate data back to orig shape
climate_pre_scaled = climate_pre_scaled.reshape(climate_pre_ogshape)

In [17]:
climate_pre_scaled = np.array(climate_pre_scaled)
sfe_pre_scaled = np.array(sfe_pre_scaled)

# wrap the lists as object arrays - allows arrays of different lengths to be saved
sfe_obj = np.array(sfe_pre_scaled, dtype=object)
climate_obj = np.array(climate_pre_scaled, dtype=object)

In [18]:
np.savez_compressed("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_3mwindow_scaled.npz", sfe_pre = sfe_obj)
np.savez_compressed("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_3mwindow_scaled.npz", climate_pre =climate_obj)

## training the model

https://www.geeksforgeeks.org/deep-learning/long-short-term-memory-lstm-rnn-in-tensorflow/

In [2]:
# reading in sliding window data, was also scaled
sfe_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_3mwindow_scaled.npz", allow_pickle = True)["sfe_pre"] 
climate_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_3mwindow_scaled.npz", allow_pickle = True)["climate_pre"]

In [3]:
# converting from objects
sfe_pre = np.array(sfe_pre, dtype=np.float32)
climate_pre = np.array(climate_pre, dtype=np.float32)


In [4]:
climate_train, climate_test, sfe_train, sfe_test = train_test_split(
    climate_pre, sfe_pre, test_size=0.1, shuffle=False
)

In [5]:
model = Sequential()
model.add(BatchNormalization(input_shape=(climate_pre.shape[1], climate_pre.shape[2]))) # input shape is the length of the sequence, each input feature
model.add(LSTM(units=64, return_sequences=False,
          input_shape=(climate_pre.shape[1], climate_pre.shape[2]))) # return one output per sequence
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics = ['mae'])
model.summary()

2025-11-30 09:40:54.128075: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-30 09:40:54.726328: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38199 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:87:00.0, compute capability: 8.0


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 3, 8)             32        
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 64)                18688     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 18,785
Trainable params: 18,769
Non-trainable params: 16
_________________________________________________________________


In [6]:
history = model.fit(climate_train, sfe_train, epochs=40, batch_size=64, validation_split=0.1)


Epoch 1/40


2025-11-30 09:41:08.382823: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


   11/48555 [..............................] - ETA: 4:14 - loss: 0.0952 - mae: 0.2484    

2025-11-30 09:41:09.112554: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


48555/48555 [==============================] - 243s 5ms/step - loss: 0.0029 - mae: 0.0395 - val_loss: 0.0021 - val_mae: 0.0337
Epoch 2/40
48555/48555 [==============================] - 241s 5ms/step - loss: 0.0023 - mae: 0.0354 - val_loss: 0.0021 - val_mae: 0.0323
Epoch 3/40
48555/48555 [==============================] - 242s 5ms/step - loss: 0.0021 - mae: 0.0342 - val_loss: 0.0020 - val_mae: 0.0330
Epoch 4/40
48555/48555 [==============================] - 242s 5ms/step - loss: 0.0020 - mae: 0.0335 - val_loss: 0.0018 - val_mae: 0.0300
Epoch 5/40
48555/48555 [==============================] - 242s 5ms/step - loss: 0.0020 - mae: 0.0331 - val_loss: 0.0018 - val_mae: 0.0300
Epoch 6/40
48555/48555 [==============================] - 243s 5ms/step - loss: 0.0019 - mae: 0.0328 - val_loss: 0.0017 - val_mae: 0.0295
Epoch 7/40
48555/48555 [==============================] - 243s 5ms/step - loss: 0.0019 - mae: 0.0325 - val_loss: 0.0018 - val_mae: 0.0305
Epoch 8/40
48555/48555 [=====================

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



48555/48555 [==============================] - 247s 5ms/step - loss: 0.0017 - mae: 0.0308 - val_loss: 0.0015 - val_mae: 0.0279
Epoch 33/40
48555/48555 [==============================] - 249s 5ms/step - loss: 0.0017 - mae: 0.0307 - val_loss: 0.0015 - val_mae: 0.0279
Epoch 34/40
48555/48555 [==============================] - 248s 5ms/step - loss: 0.0017 - mae: 0.0307 - val_loss: 0.0015 - val_mae: 0.0280
Epoch 35/40
48555/48555 [==============================] - 250s 5ms/step - loss: 0.0017 - mae: 0.0307 - val_loss: 0.0016 - val_mae: 0.0286
Epoch 36/40
48555/48555 [==============================] - 249s 5ms/step - loss: 0.0017 - mae: 0.0307 - val_loss: 0.0016 - val_mae: 0.0283
Epoch 37/40
48555/48555 [==============================] - 248s 5ms/step - loss: 0.0017 - mae: 0.0307 - val_loss: 0.0016 - val_mae: 0.0285
Epoch 38/40
48555/48555 [==============================] - 247s 5ms/step - loss: 0.0017 - mae: 0.0306 - val_loss: 0.0015 - val_mae: 0.0277
Epoch 39/40
48555/48555 [==============

NameError: name 'scaler_sfe' is not defined

In [7]:
model.save("/scratch/users/ashdef/model_out/3month_dropout/baseline.keras")


In [ ]:
history.history

In [8]:
np.save("/scratch/users/ashdef/model_out/3month_dropout/training_history.npy", history.history)

In [19]:
predictions = model.predict(climate_test)
np.save("/scratch/users/ashdef/model_out/3month_dropout/predictions_scaled.npy", predictions)

predictions_original = scaler_sfe.inverse_transform(predictions)
np.save("/scratch/users/ashdef/model_out/3month_dropout/predictions_original.npy", predictions_original)

np.save("/scratch/users/ashdef/model_out/3month_dropout/sfe_test_scaled.npy", sfe_test)
    
sfe_test_original = scaler_sfe.inverse_transform(sfe_test)
np.save("/scratch/users/ashdef/model_out/3month_dropout/sfe_test_original.npy", sfe_test_original)

11989/11989 [==============================] - 25s 2ms/step


In [20]:
rmse = mean_squared_error(sfe_test_original, predictions_original, squared=False)
mae = mean_absolute_error(sfe_test_original, predictions_original)
r2 = r2_score(sfe_test_original, predictions_original)

print(rmse, mae, r2)

0.2101121 0.15265065 0.9673721539116159
